# Data to Pandas table

**Purpose of script:**

Put mw and opt data to one pandas dataframe, create features. One row per pixel.

- In: opt and mw data, all files
- Out: one file (maybe several due to size constraints) (csv or parquet) with table of features prepared to be used in model

In [70]:
#import matplotlib.pyplot as plt
#import numpy as np

# import rioxarray
import xarray
# import rasterio
# import gdal

# from os import listdir
# from os.path import isfile, join

#%matplotlib inline

import pandas as pd
from datetime import datetime

Relevant paths:

In [5]:
mw_path = r"../Data/microwave-rs/mw_interpolated/2019-07-01_mw.tif"
opt_path = r"../Data/optical-rs/2019-07-01_grain_diameter.tif"

Load data:

In [ ]:
# load all files
# put to one xarray?

In [72]:
#TEMP Data load:
data_opt = xarray.open_dataarray(opt_path)
data_mw = xarray.open_dataarray(mw_path)

To pandas dataframe:

In [ ]:
# for loop

In [124]:
# convert mw to pandas
df_mw = data_mw.to_dataframe()
df_opt = data_opt.to_dataframe()
# fix index
df_mw = df_mw.reset_index()
# remove columns: spacial_ref, band
df_mw = df_mw[['x', 'y', 'band_data']]
# rename
df_mw.rename({'band_data': 'mw_value'}, axis=1, inplace=True)
# ----------------------
# convert opt to pandas
df_opt = data_opt.to_dataframe()
# fix index
df_opt = df_opt.reset_index()
# remove columns: spacial_ref, band
df_opt = df_opt[['x', 'y', 'band_data']]
# rename
df_opt.rename({'band_data': 'opt_value'}, axis=1, inplace=True)
# ----------------------
# merge
df_comb = pd.merge(df_mw, df_opt, how = 'left', on = ['y', 'x']) # left smaller mw, right - opt
# fill na for masked opt data
df_comb.opt_value.fillna(-1, inplace=True)

Add features:

In [126]:
# add column and row (to add neighbours)
df_comb['col'] = df_comb.groupby("x").ngroup() # xshape 2663 
df_comb['row'] = df_comb.groupby("y").ngroup(ascending=False) # yshape 1462

In [128]:
# remove water:
df_comb = df_comb.loc[df_comb['mw_value'] != -1]

In [ ]:
# add date:
df_comb['date'] = '2019-07-01'
df_comb["date"]= pd.to_datetime(df_comb["date"])
# df_comb['month'] = df_comb["date"].dt.month

# need to suppress warning

In [ ]:
# add neighbours (3x3 val num from 1 to 9):
# based on row and col value??

df_comb['val_1'] = 0



In [ ]:
# add neighbours (for mw, if -1 then nan?)
# add pooled
# add if coast column - if at least one na but not all 

In [ ]:
# convert x and y to useful sections of geodata

Write to csv/ parquet: (?)

In [113]:
# write to csv
df_comb.to_csv('melt_2019-07-01.csv', index= False)